In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
import time
import urllib3
import json

In [3]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    urllib3.disable_warnings()
    parser = html.fromstring(resp)


    return parser

def links_proyectos(parser):

    links1 = parser.xpath('//div[@class="block5_widget_content row-2-col"]//a/@href')
    
    urls1 = []
    for link in links1:
        if link.startswith('https://'):
            urls1.append(link.strip())

    urls1 = list(set(urls1))

    links_bottons= parser.xpath('//div[@class="wpsm_panel-group"]//a/@href')

    urls2 = []
    for link in links_bottons:
        if link.startswith('https://'):
            urls2.append(link.strip())

    urls2 = list(set(urls2))

    urls1.extend(urls2)

    return urls1

In [4]:
### Titulo

def titulo(parser_proy):
    
    titulo = parser_proy.xpath('//header/h1[@class="entry-title"]/text()')

    return titulo[0]

def descripcion(parser_proy):
    descripcion = parser_proy.xpath('//div[@class="pf-content"]//p/text()')

    desc_completa = ''
    for desc in descripcion:
        desc = desc.strip()
        if len(desc) != 0: 
            desc_completa = desc_completa + desc + ' '

    return desc_completa.strip()

### Ano convocatoria

def ano(parser_proy):
    ano = parser_proy.xpath('//script[@class="yoast-schema-graph"]/text()')
    diccionario = json.loads(ano[0])
    ano = diccionario['@graph'][2]['datePublished'].split('-')[0]

    return ano

# Ganadores
def f_ganadores_proyecto(parser_proy):
    fechas = parser_proy.xpath('//p[contains(@class, "has-text-color")]//strong/text()')

    ganadores = ''
    for fecha in fechas:
        fecha = fecha.strip('• ')
        
        if fecha.startswith('Publicación'):
            ganadores = fecha.split(': ')[1]

    return ganadores

## Realización
def f_periodo_realizacion_proyecto(parser_proy):
    fechas = parser_proy.xpath('//p[contains(@class, "has-text-color")]//strong/text()')

    realizacion = ''
    for fecha in fechas:
        fecha = fecha.strip('• ')

        if fecha.startswith('Realización'):
            realizacion = fecha.split(': ')[1]


    return realizacion

## Informe Final
def f_entrega_informe_final(parser_proy):  
    fechas = parser_proy.xpath('//p[contains(@class, "has-text-color")]//strong/text()')

    informe_final = ''
    for fecha in fechas:
        fecha = fecha.strip('• ')
        if fecha.startswith('Presentación'):
            informe_final = fecha.split(': ')[1]

    return informe_final
# Fecha cierre
def f_cierre_inscripcion(parser_proy):
    fechas = parser_proy.xpath('//p[contains(@class, "has-text-color")]//strong/text()')
    
    cierre = ''

    try:
        for fecha in fechas:
            fecha = fecha.strip('• ').lower()
            
            if fecha.startswith('Cierre') or 'cierra' in fecha or 'cierre' in fecha:
                cierre = fecha
    except IndexError:
        cierre = ''

    return cierre.strip()


#PDFs
def pdfs(parser_proy):
    links = parser_proy.xpath('//a/@href')
    urls = ''
    for link in links:
        link = link.strip()
        if link.endswith('.pdf'):
            urls = urls + link + ', '

    return urls.rstrip(', ')



In [5]:
## Parser semilla y Links Proyectos

def clacso():
    parsert = parser('https://www.clacso.org/convocatorias/')
    links_proy = links_proyectos(parsert)

    links_proy.remove("https://www.clacso.org/category/concursos-y-convocatorias/convocatorias-2019/")
    links_proy.remove("https://www.clacso.org/category/concursos-y-convocatorias/convocatorias-2017/")
    links_proy.remove("https://www.clacso.org/category/concursos-y-convocatorias/convocatorias-2018/")
    links_proy.remove("https://www.clacso.org/category/concursos-y-convocatorias/convocatorias-2020/")

    clacso = pd.DataFrame()

    titulos = []
    anos = []
    descripciones = []
    cierres = []
    ganadores = []
    realizaciones = []
    informes_finales = []
    urls = []

    for link in links_proy:

        parser_proy = parser(link)

        titulos.append(titulo(parser_proy))
        anos.append(ano(parser_proy))
        descripciones.append(descripcion(parser_proy))
        cierres.append(f_cierre_inscripcion(parser_proy))
        ganadores.append(f_ganadores_proyecto(parser_proy))
        realizaciones.append(f_periodo_realizacion_proyecto(parser_proy))
        informes_finales.append(f_periodo_realizacion_proyecto(parser_proy))
        urls.append(pdfs(parser_proy))

    clacso['Titulo'] = titulos
    clacso['Descripción'] = descripciones
    clacso['Ano'] = anos
    clacso['Cierre inscripción'] = cierres
    clacso['Ganadores'] = ganadores
    clacso['Periodo de realización'] = realizaciones
    clacso['Entrega informe final'] = informes_finales
    clacso['Link proyecto'] = links_proy
    clacso['Pdfs'] = urls

    return clacso



In [6]:
clacso = clacso()

clacso.to_excel('Clacso.xlsx')

/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.clacso.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,Titulo,Descripción,Ano,Cierre inscripción,Ganadores,Periodo de realización,Entrega informe final,Link proyecto,Pdfs
0,Becas CLACSO-UNQ para cursar estudios de maestría,Ciclo de formación 2021 Con el propósito de fo...,2020,,,,,https://www.clacso.org/becas-clacso-unq-para-c...,
1,Feminismo y ambiente: un campo emergente en lo...,Las relaciones entre los derechos de las mujer...,2020,"cierre de inscripción: 8 de febrero de 2021, 2...",febrero de 2021,marzo-junio de 2021,marzo-junio de 2021,https://www.clacso.org/feminismo-y-ambiente-un...,
2,Mes de la mujer: 100 becas de formación para m...,La Secretaría Ejecutiva de CLACSO a través de ...,2021,,,,,https://www.clacso.org/mes-de-la-mujer-100-bec...,
3,Becas CLACSO-UNQ para cursar estudios de maestría,La Universidad Nacional de Quilmes y el Consej...,2019,,,,,https://www.clacso.org/becas-clacso-unq-para-c...,https://www.clacso.org.ar/concursos_adm/adjunt...
4,"Convocatoria de Ensayos “Malvinas, Antártida y...","La Secretaría de Malvinas, Antártida y Atlánti...",2020,nueva fecha de cierre: 10 de agosto,,,,https://www.clacso.org/convocatoria-de-ensayos...,http://biblioteca.clacso.edu.ar/clacso/se/2020...
5,Concurso Latinoamericano de Ensayo sobre Acces...,"AmeliCA, UNESCO, Redalyc y CLACSO, con motivo ...",2020,,,,,https://www.clacso.org/concurso-latinoamerican...,
6,Concurso para la selección de seminarios virtu...,a) Realizar un artículo analítico que ponga en...,2020,cierre de la convocatoria 30 de noviembre de 2020,,,,https://www.clacso.org/concurso-para-la-selecc...,
7,Concurso Nacional de Ensayos Académicos «Pasad...,Fecha límite de presentación: 17/10/2019 Con m...,2019,,,,,https://www.clacso.org/concurso-nacional-de-en...,http://www.sociales.uba.ar/wp-content/blogs.di...
8,Becas CLACSO-UNQ para cursar estudios de maestría,– Con el propósito de fortalecer la\nformación...,2019,,,,,https://www.clacso.org/becas-clacso-unq-para-c...,
9,Concurso para la selección de seminarios virtu...,"Para Redes de Posgrado, Grupos de Trabajo y Ce...",2019,cierre de la convocatoria 2 de febrero de 2020,,,,https://www.clacso.org/concurso-para-la-selecc...,
